In [6]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from ClassroomSim.simulate_one_classroom import droplet_risk_by_distance

In [12]:
import numpy as np 

def droplet_risk_by_distance(
    d: float, 
    susceptible_status: str, 
    source_status: str, 
    time: float, 
    class_type: str, 
    VE_params: dict
) -> float:

    r"""
    Compute the risk that an unmasked susceptible person is infected through 
    droplet transmission by an unmasked source case seated a certain distance 
    away from him/her, depending on the vaccination status of both individuals, 
    exposure time, VE parameters, and class type. (The risk of droplet 
    transmission decreases with distance.)
    
    Args:
        d: distance between source and susceptible, measured in meters.
        susceptible status: 'V' = vaccinated, 'U' = unvaccinated
        source_status: 'V' = vaccinated, 'U' = unvaccinated
        time: length of exposure, measured in hours
        pixels_per_foot: a parameter for translating distance in feet to 
            the number of pixels in the room layout
        class_type: one of ['breathing', 'speaking', 'singing', 
            'heavy_breathing']
        VE_params: dictionary for vaccine effectiveness parameters. 
            'VE_susceptible' = reduction in infection risk for a vaccinated 
                susceptible person
            'VE_transmission' = reduction in the viral load emitted by a 
                vaccinated infectious person
    Returns:
        combined risk of droplet and aerosol transmission
    """

    # convert distance in pixels to distance in meters
    susceptible_const = 1
    source_const = 1

    if susceptible_status == 'V':
        # probabiltiy of infection of a susceptible individual is scaled by 
        # susceptible_const if he/she is vaccinated
        susceptible_const = 1 - VE_params['VE_susceptible']
    if source_status == 'V':
        # viral load of the source is reduced by source_const if vaccinated
        source_const = (1 - VE_params['VE_transmission'])

    if class_type == 'speaking':
        source_const = source_const*6
    elif class_type == 'singing':
        source_const = source_const*48
    elif class_type == 'heavy_breathing':
        source_const = source_const*15

    droplet_transmission_prob = 2.4 * susceptible_const * \
        (1-np.exp(-1 * source_const*0.0135*time*(-0.1819*np.log(d)+0.43276)/d))

    droplet_transmission_prob = float(droplet_transmission_prob)
    return max(droplet_transmission_prob, 0)


In [23]:
aerosol_params = {
    'inhale_air_rate': 6.8,
    'dose_response_constant': 1440,
    # "nominal" corresponding to viral load of 10^8 copies / mL
    'nominal_breathe_virus_emitted_hourly': 3300, 
    'nominal_talk_virus_emitted_hourly' : 27300,
    'nominal_sing_virus_emitted_hourly' : 330000,
    'nominal_heavy_breathe_virus_emitted_hourly': 3300*15,
    # distribution of viral load over orders of magnitude from 10^5 to 10^11
    'viral_load_distribution': [0.12, 0.22, 0.3, 0.23, 0.103, 0.0236, 0.0034]    
}


def aerosol_risk(
    room_vol: float, 
    source_is_vax: int, 
    time: float, 
    class_type: str, 
    VE_params: dict, 
    aerosol_params: dict
) -> float:
    r"""
    Compute the risk due to aerosol transmission for an unvaccinated 
    susceptible person. The risk of aerosol transmission is assumed to be 
    uniform over all distances.
    
    Args:
        room_vol: volume of the room, measured in cubic meters
        source_is_vax: 1 if the source is vaccinated, 0 otherwise
        time: exposure time, measured in hours
        class_type: one of ['breathing', 'speaking', 'singing', 
            'heavy_breathing', 'no_aerosol']. The 'no_aerosol' scenario applies 
            to outdoor settings, where aerosols are quickly diluted by airflow.
        VE_params: dictionary of vaccine effectiveness parameters. 
            'VE_susceptible' = amount reduction in infection risk for a 
                vaccinated susceptible person. e.g., VE_susceptible = 0.7 
                <=> infection risk gets multiplied by 0.3 
            'VE_transmission' = amount reduction in the viral load emitted by a 
                vaccinated infectious person. e.g., VE_transmission = 0.7 
                <=> emitted viral load gets multiplied by 0.3
        aerosol_params: dict of parameters for the aerosol transmission model
    Returns:
        risk of aerosol transmission
    """

    if class_type == 'breathing':
        v = aerosol_params['nominal_breathe_virus_emitted_hourly']
    elif class_type == 'speaking':
        v = aerosol_params['nominal_talk_virus_emitted_hourly']
    elif class_type == 'singing':
        v = aerosol_params['nominal_sing_virus_emitted_hourly']
    elif class_type == 'heavy_breathing':
        v = aerosol_params['nominal_heavy_breathe_virus_emitted_hourly']
    elif class_type == 'no_aerosol':
        return 0

    hourly_virus_array = np.array([v/1000, v/100, v/10, v, v*10, v*100, v*1000])
    dose_array = hourly_virus_array * aerosol_params['inhale_air_rate']/room_vol

    if source_is_vax == 1:
        dose_array = dose_array * (1 - VE_params['VE_transmission'])

    effective_dose_array = dose_array / aerosol_params['dose_response_constant']
    unvax_susceptible_risk_array = 1 - np.exp(-effective_dose_array)
    unvax_susceptible_risk = np.dot(
        unvax_susceptible_risk_array, 
        np.array(aerosol_params['viral_load_distribution'])
    )
    # scale by 2.4 for Delta
    unvax_susceptible_risk_over_time = 2.4*unvax_susceptible_risk * time 

    return unvax_susceptible_risk_over_time

In [15]:
import pickle

def load_rooms(distancing):
    if distancing == 1:
        with open("../Data/small_room_info.pickle" , 'rb') as handle:
            layout = pickle.load(handle)
            pixels_per_foot = layout['pixels_per_foot']
            room_vol = layout['volume']
            room = layout['plan']

    elif distancing == 3:
        with open("../Data/medium_room_info.pickle" , 'rb') as handle:
            layout = pickle.load(handle)
            pixels_per_foot = layout['pixels_per_foot']
            room_vol = layout['volume']
            room = layout['plan']

    else: 
        with open("../Data/big_room_info.pickle" , 'rb') as handle:
            layout = pickle.load(handle)
            pixels_per_foot = layout['pixels_per_foot']
            room_vol = layout['volume']
            room = layout['plan']
    
    return room, room_vol, pixels_per_foot

In [19]:
for distancing in [1,3,6]:
    _, room_vol, _ = load_rooms(distancing)
    print(distancing, room_vol)


1 222004
3 1087931
6 1684283


In [20]:
aerosol_risk(
    room_vol=222004,
    source_is_vax=0,
    time=1,
    class_type='breathing',
    VE_params={'VE_susceptible': 0., 'VE_transmission': 0.},
    aerosol_params=aerosol_params
)

0.0011669770604804815

In [13]:
droplet_risk_by_distance(
    d=1, susceptible_status="U", source_status="U", time=1, class_type="breathing", 
    VE_params={"VE_susceptible": 0, "VE_transmission": 0})


0.013980545244559295

In [28]:
for d in np.linspace(0,10,21):
    droplet_risk = droplet_risk_by_distance(
        d=d, susceptible_status="U", source_status="V", time=1, class_type="breathing", 
        VE_params={"VE_susceptible": 0.5, "VE_transmission": 0.5})
    aerosol_risk_ = aerosol_risk(
            room_vol=222004,
            source_is_vax=1,
            time=1,
            class_type='breathing',
            VE_params={'VE_susceptible': 0.5, 'VE_transmission': 0.5},
            aerosol_params=aerosol_params
        )
    print(d, droplet_risk, aerosol_risk_, droplet_risk+aerosol_risk_)

0.0 2.4 0.0005887046186657633 2.4005887046186656
0.5 0.01803839860990202 0.0005887046186657633 0.018627103228567782
1.0 0.007000482362552329 0.0005887046186657633 0.007589186981218092
1.5 0.003874133460514972 0.0005887046186657633 0.004462838079180735
2.0 0.00248279476637725 0.0005887046186657633 0.003071499385043013
2.5 0.0017236226900277706 0.0005887046186657633 0.0023123273086935337
3.0 0.0012574515648016594 0.0005887046186657633 0.0018461561834674225
3.5 0.0009481255681154543 0.0005887046186657633 0.0015368301867812175
4.0 0.0007312904393084629 0.0005887046186657633 0.0013199950579742261
4.5 0.0005729375596411401 0.0005887046186657633 0.0011616421783069034
5.0 0.0004535676451936865 0.0005887046186657633 0.0010422722638594498
5.5 0.00036128088057036934 0.0005887046186657633 0.0009499854992361327
6.0 0.00028844783670693984 0.0005887046186657633 0.000877152455372703
6.5 0.00022997710083414444 0.0005887046186657633 0.0008186817194999077
7.0 0.00018235634190491012 0.0005887046186657633 

/tmp/ipykernel_1507186/1661199126.py:57: RuntimeWarning: divide by zero encountered in log
  (1-np.exp(-1 * source_const*0.0135*time*(-0.1819*np.log(d)+0.43276)/d))
